In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [ ]:
train = train.drop(['location'], axis=1)

train.isna().sum()

id          0
keyword    61
text        0
target      0
dtype: int64

In [ ]:
train.isna().sum()

id          0
keyword    61
text        0
target      0
dtype: int64

In [ ]:
import re

def remove_link(text):
    return re.sub('https?://\S+|www\.\S+', '', text)

train['text'] = train.text.apply(remove_link)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(train):
  data = []
  for i in range(train.shape[0]):
    text = re.sub('[^a-zA-Z]', '', train.iloc[i]['text'])
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    all_stepwords = stopwords.words('english')
    all_stepwords.remove('not')
    text = [ps.stem(word) for word in text if not word in set(all_stepwords)]
    text = ' '.join(text)
    data.append(text)
  return data

corpus = remove_stopwords(train)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000)

x = cv.fit_transform(corpus).toarray()
y = train['target'].values

print(x.shape, y.shape)

(7613, 2000) (7613,)


In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [ ]:
lr.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
test = pd.read_csv('test.csv')

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [ ]:
test.shape

(3263, 4)

In [ ]:
test.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [ ]:
test = test.drop(['location'], axis=1)

test.isna().sum()

id          0
keyword    26
text        0
dtype: int64

In [ ]:
test['text'] = test.text.apply(remove_link)

In [ ]:
corpus2 = remove_stopwords(test)

In [ ]:
x_test = cv.transform(corpus2)

In [ ]:
#score of logistic regression model
lr.score(x, y)

0.7443846052804414

In [ ]:
y_test = lr.predict(x_test)

In [ ]:
y_test

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

In [ ]:
dtc.fit(x, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dtc.score(x, y)

0.7443846052804414

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=20)

In [ ]:
rfc.fit(x, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rfc.score(x, y)

0.7443846052804414

In [ ]:
sub = pd.read_csv('sample_submission.csv')

sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
test.head()

,id,keyword,text
0,0,NaN,Just happened a terrible car crash
1,2,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
id_val = sub['id']
target = y_test

In [ ]:
df = pd.DataFrame({'id': id_val, 'target': target})

In [ ]:
df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [ ]:
df.to_csv('submission.csv', index=False)

In [ ]:
from google.colab import files

files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>